#### 分层的 textCNN 模型。title部分不变，content部分使用分层。 ** 句子层面是 23457 的，doc 层面是 124**

- doc_len: 每个文档的句子数；
- sent_len: 每个句子的词数量；
- embedding_size: 词向量长度。
- n_filter:每种filter的个数
- filter_num_sent：sent中不同长度的filter种数
- filter_num_doc：不同长度的filter种数

X_inputs.shape=[batch_size, doc_len, sent_len] ->(0.embedding)-> sent_inputs.shape=[batch_size, doc_len, sent_len, embedding_size] 

sentence_encoder:每个句子得到一个向量:<br/>
sent_inputs ->(reshape)-> [batch_size×doc_len, sent_len, embedding_size] ->(textCNN)-> doc_inputs:[batch_size×doc_len, n_filter*filter_num_sent]

doc_encoder: 每个文档得到一个向量：<br/>
doc_inputs:[batch_size×doc_len, n_filter*filter_num] ->(reshpe)-> [batch_size, doc_len, n_filter_sent*filter_num] ->(TextCNN)-> doc_outputs:[batch_size, n_filter*filter_num_doc]

title部分输出：[batch_size, n_filter*filter_num_sent]  

两部分的输出进行拼接，然后加 fc+bn+relu+dropout

sigmoig_output_layer.

参考代码： 

https://github.com/yongyehuang/deep-text-classifier/blob/master/HAN_model.py 其中attention最后加权的地方应该写错了

https://github.com/indiejoseph/doc-han-att/blob/master/model.py

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import os
import sys
import shutil
import time

time0 = time.time()
print('Starting ...')
model_name = 'han-cnn-title-content-256-23457-1234'                    # 模型名称
W_embedding = np.load('../data/W_embedding.npy')            # 导入预训练好的词向量
model_path = '../ckpt/' + model_name + '/'                  # 模型保存位置
summary_path = '../summary/' + model_name + '/'             # summary 位置
result_path = '../result/' + model_name + '.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '.npy'            # scores.npy 位置
save_path = '../data/'

if not os.path.exists(model_path):
    os.makedirs(model_path)         
model_path = model_path + 'model.ckpt'
if os.path.exists(summary_path):   # 删除原来的 summary 文件，避免重合
    shutil.rmtree(summary_path)
os.makedirs(summary_path)          # 然后再次创建

# ##################### config ######################
title_len = n_step1 = 50                  # title 长度
sent_len = n_step2 = 40                 # 每个句子的词数
doc_len = n_step3 = 10                   # 每个文档的句子数

filter_sizes_sent = [2,3,4,5,7]
filter_sizes_doc = [1,2,3,4]
n_filter = 256
fc_hidden_size = 1024                   # fc 层节点数
input_size = embedding_size = 256       # 字向量长度
n_class = 1999
n_layer = 2        # bi-gru 层数
max_grad_norm = 5.0  # 最大梯度（超过此值的梯度将被裁剪）
global_step = 0
valid_num = 100000
seed_num = 13
te_batch_size = 256 
tr_batch_size = 256 
print('Prepared, costed time %g s.' % (time.time() - time0))

Starting ...
Prepared, costed time 0.328925 s.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers

'''
HAN 模型，知乎问题多标签分类。
'''

print('Building model ...')
lr = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32, [])
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
tst = tf.placeholder(tf.bool)
n_updates = tf.placeholder(tf.int32)      # training iteration,传入 bn 层
update_emas = list()   # BN 层中所有的更新操作

def weight_variable(shape, name):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def batchnorm(Ylogits, is_test, num_updates, offset, convolutional=False):
    """batchnormalization.
    Args:
        Ylogits: 1D向量或者是3D的卷积结果。
        num_updates: 迭代的global_step
        offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
        scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
        m: 表示batch均值；v:表示batch方差。
        bnepsilon：一个很小的浮点数，防止除以 0.
    Returns:
        Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
        update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
    """
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, num_updates) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

def textcnn(X_inputs, n_step, filter_sizes, embed_size):
    """build the TextCNN network. Return the h_drop"""
    inputs = tf.expand_dims(X_inputs, -1)
    pooled_outputs = list()
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            # Convolution Layer
            filter_shape = [filter_size, embed_size, 1, n_filter]
            W_filter = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W_filter")
#             tf.summary.histogram('W_filter', W_filter)
            beta = tf.Variable(tf.constant(0.1, tf.float32, shape=[n_filter], name="beta"))
            tf.summary.histogram('beta', beta)
            conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
            conv_bn, update_ema = batchnorm(conv, tst, n_updates, beta, convolutional=True)    # 在激活层前面加 BN
            # Apply nonlinearity, batch norm scaling is not useful with relus
            # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
            h = tf.nn.relu(conv_bn, name="relu")
            # Maxpooling over the outputs
            pooled = tf.nn.max_pool(h,ksize=[1, n_step - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],padding='VALID',name="pool")
            pooled_outputs.append(pooled)
            update_emas.append(update_ema)
    h_pool = tf.concat(pooled_outputs, 3)
    n_filter_total = n_filter * len(filter_sizes)
    h_pool_flat = tf.reshape(h_pool, [-1, n_filter_total]) 
    return h_pool_flat    # shape = [-1, n_filter_total]    
    
with tf.name_scope('Inputs'):
    X1_inputs = tf.placeholder(tf.int64, [None, title_len], name='X1_inputs')
    X2_inputs = tf.placeholder(tf.int64, [None, doc_len*sent_len], name='X2_inputs')
    y_inputs = tf.placeholder(tf.float32, [None, n_class], name='y_input')    

with tf.name_scope('embedding'):
    embedding = tf.get_variable(name='embedding', shape=W_embedding.shape, 
                                initializer=tf.constant_initializer(W_embedding), trainable=True)   # fine-tune
    
def cnn_reference(X_inputs):
    """TextCNN 模型。title部分。
    Args: 
        X_inputs: tensor.shape=(batch_size, title_len)
    Returns:
        title_outputs: tensor.shape=(batch_size, n_filter*filter_num_sent)
    """
    inputs = tf.nn.embedding_lookup(embedding, X_inputs)
    with tf.variable_scope('title_encoder'):  # 生成 title 的向量表示
        title_outputs = textcnn(inputs, n_step1, filter_sizes_sent, embed_size=embedding_size)
    return title_outputs   # shape = [batch_size, n_filter*filter_num_sent]
    
def han_reference(X_inputs):
    """分层 TextCNN 模型。content部分。
    Args: 
        X_inputs: tensor.shape=(batch_size, doc_len*sent_len)
    Returns:
        doc_attn_outputs: tensor.shape=(batch_size, n_filter*filter_num_doc)
    """
    inputs = tf.nn.embedding_lookup(embedding, X_inputs)    # inputs.shape=[batch_size, doc_len*sent_len, embedding_size]
    sent_inputs = tf.reshape(inputs,[batch_size*doc_len, sent_len, embedding_size]) # [batch_size*doc_len, sent_len, embedding_size]
    with tf.variable_scope('sentence_encoder'):  # 生成句向量
        sent_outputs = textcnn(sent_inputs, n_step2, filter_sizes_sent, embed_size=embedding_size)
    with tf.variable_scope('doc_encoder'):  # 生成文档向量
        doc_inputs = tf.reshape(sent_outputs, [batch_size, doc_len, n_filter*len(filter_sizes_sent)]) # [batch_size, doc_len, n_filter*len(filter_sizes_sent)]
        doc_outputs = textcnn(doc_inputs, n_step3, filter_sizes_doc, n_filter*len(filter_sizes_sent))  # [batch_size, doc_len, n_filter*filter_num_doc]
    return doc_outputs    # [batch_size,  n_filter*len(filter_sizes_sent)]
        

with tf.name_scope('title_cnn'):
    output_title = cnn_reference(X1_inputs)
with tf.name_scope('content_han'):
    output_content = han_reference(X2_inputs)   
    
with tf.name_scope('fc-bn-layer'):
    output = tf.concat([output_title, output_content], axis=1)
    output_size = n_filter*(len(filter_sizes_sent) + len(filter_sizes_doc))
    W_fc = weight_variable([output_size, fc_hidden_size], name='Weight_fc')
    tf.summary.histogram('W_fc', W_fc)
    h_fc = tf.matmul(output, W_fc, name='h_fc')
    beta_fc = tf.Variable(tf.constant(0.1, tf.float32, shape=[fc_hidden_size], name="beta_fc"))
    tf.summary.histogram('beta_fc', beta_fc)
    fc_bn, update_ema_fc = batchnorm(h_fc, tst, n_updates, beta_fc, convolutional=False)
    update_emas.append(update_ema_fc)
    fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
    fc_bn_drop = tf.nn.dropout(fc_bn_relu, keep_prob)

with tf.name_scope('out_layer'):
    W_out = weight_variable([fc_hidden_size, n_class], name='Weight_out') 
    tf.summary.histogram('Weight_out', W_out)
    b_out = bias_variable([n_class], name='bias_out') 
    tf.summary.histogram('bias_out', b_out)
    y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  #每个类别的分数 scores
    
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_inputs))
    tf.summary.scalar('cost', cost)
    
# ***** 优化求解 *******
with tf.name_scope('AdamOptimizer'):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    train_op = optimizer.apply_gradients( zip(grads, tvars),
        global_step=tf.contrib.framework.get_or_create_global_step())
    update_op = tf.group(*update_emas)   # 更新 BN 参数

merged = tf.summary.merge_all() # summary
train_writer = tf.summary.FileWriter(summary_path + 'train', sess.graph)
test_writer = tf.summary.FileWriter(summary_path + 'test')
print 'Finished creating the TextCNN model.'

Building model ...
Finished creating the TextCNN model.


## 导入数据

In [3]:
import sys
sys.path.append('..')
from data_helpers import BatchGenerator
from data_helpers import to_categorical
from evaluator import score_eval


print('loading data...')
time0 = time.time()
X_title = np.load(save_path+'X_tr_title_50.npy') 
sample_num = X_title.shape[0]
X_content = np.load(save_path+'train_segs_content.npy')
X_content = X_content.reshape([-1, doc_len*sent_len])
X = np.hstack([X_title, X_content])
y = np.load(save_path+'y_tr.npy')
print('finished loading data, time cost %g s' % (time.time() - time0))
# 划分验证集
np.random.seed(seed_num)
new_index = np.random.permutation(sample_num)
X = X[new_index]
y = y[new_index]
X_valid = X[:valid_num]
y_valid = y[:valid_num]
X_train = X[valid_num:]
y_train = y[valid_num:]
print('train_num=%d, valid_num=%d' % (X_train.shape[0], X_valid.shape[0]))

# 构建数据生成器
data_train = BatchGenerator(X_train, y_train, shuffle=True)
data_valid = BatchGenerator(X_valid, y_valid, shuffle=False)

print('X_train.shape=', X_train.shape, 'y_train.shape=', y_train.shape)
print('X_valid.shape=', X_valid.shape, 'y_valid.shape=', y_valid.shape)

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
valid_data_size = data_valid.y.shape[0]
def valid_epoch():
    """Testing or valid."""
    data_valid._index_in_epoch = 0  # 先指向第一个值
    _batch_size = te_batch_size
    fetches = [cost, y_pred]   
    batch_num = int(valid_data_size / _batch_size)
    start_time = time.time()
    _costs = 0.0
    predict_labels_list = list()  # 所有的预测结果
    for i in xrange(batch_num):
        X_batch, y_batch = data_valid.next_batch(_batch_size)
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, y_inputs:y_batch, lr:1e-5,
                    batch_size:tr_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        _cost, predict_labels = sess.run(fetches, feed_dict)
        _costs += _cost    
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    mean_cost = _costs / batch_num
    return mean_cost, precision, recall, f1

print('Finised loading data, time %g s' % (time.time() - time0))

loading data...
finished loading data, time cost 107.719 s
train_num=2899952, valid_num=100000
('X_train.shape=', (2899952, 450), 'y_train.shape=', (2899952,))
('X_valid.shape=', (100000, 450), 'y_valid.shape=', (100000,))
Finised loading data, time 127.809 s


## 训练模型

In [4]:
_lr = 5e-4
max_epoch = 1
max_max_epoch = 10  # 本例中，
decay = 0.80
decay_step = 6000        # 每 valid_step 就进行一次 valid 运算
valid_step = 6000
save_epoch = 1           # 每迭代 save_epoch 次保存一次模型
tr_batch_num = int(data_train.y.shape[0] / tr_batch_size)  # 每个 epoch 中包含的 batch 数
saver = tf.train.Saver(max_to_keep=10)           # 最多保存的模型数量
sess.run(tf.global_variables_initializer())

for epoch in xrange(max_max_epoch):
    if epoch >= max_epoch:
        _lr = _lr * decay
    print 'EPOCH %d， lr=%g' % (epoch+1, _lr)
    time0 = time.time()
    _costs = 0.0
    for batch in xrange(tr_batch_num): 
        global_step += 1
        if (batch+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g ' % (
                    global_step, valid_cost, precision, recall, f1))
            if (epoch >= max_epoch):
                _lr = _lr * decay
                print('_lr = %g' % _lr)
        X_batch, y_batch = data_train.next_batch(tr_batch_size)
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:_lr, 
                     batch_size:tr_batch_size, keep_prob:0.5, tst:False, n_updates:global_step}
        fetches = [merged, cost, train_op, update_op]
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        _costs += _cost
        if global_step % 100:
            train_writer.add_summary(summary, global_step)
            X_batch, y_batch = data_valid.next_batch(tr_batch_size)
            X1_batch = X_batch[:, :title_len]
            X2_batch = X_batch[:, title_len:]
            y_batch = to_categorical(y_batch)
            feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:1e-5,
                         batch_size:tr_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)
    valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
    mean_cost = _costs / tr_batch_num
    print('global_step=%d: training cost=%g; valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
        global_step, mean_cost, valid_cost, precision, recall, f1, time.time()-time0) )
    if (epoch + 1) % save_epoch == 0: # 每 2 个 epoch 保存一次模型
        model_save_path = saver.save(sess, model_path, global_step=(epoch+1))
        print('the save path is ', model_save_path)        

EPOCH 1， lr=0.0005
global_step=6000: valid cost=0.00496794; p=1.19062, r=0.487897, f1=0.346079 
global_step=11327: training cost=0.00747537; valid cost=0.00433788; p=1.3065, r=0.533906, f1=0.379019; speed=7064.88 s/epoch
('the save path is ', '../ckpt/han-cnn-title-content-256-23457-1234/model.ckpt-1')
EPOCH 2， lr=0.0004
global_step=17327: valid cost=0.00430803; p=1.34591, r=0.550172, f1=0.390532 
_lr = 0.00032
global_step=22654: training cost=0.00409545; valid cost=0.00414844; p=1.37135, r=0.560237, f1=0.397746; speed=6403.13 s/epoch
('the save path is ', '../ckpt/han-cnn-title-content-256-23457-1234/model.ckpt-2')
EPOCH 3， lr=0.000256
global_step=28654: valid cost=0.00407105; p=1.39065, r=0.567265, f1=0.402912 
_lr = 0.0002048
global_step=33981: training cost=0.00381197; valid cost=0.00390486; p=1.39782, r=0.570365, f1=0.405078; speed=6239.69 s/epoch
('the save path is ', '../ckpt/han-cnn-title-content-256-23457-1234/model.ckpt-3')
EPOCH 4， lr=0.00016384
global_step=39981: valid cost

KeyboardInterrupt: 

In [6]:
valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
     valid_cost, precision, recall, f1, time.time()-time0) )

valid cost=0.00429876; p=1.36063, r=0.552703, f1=0.393044; speed=5141.38 s/epoch


In [9]:
print(epoch, _lr)

(3, 0.00018062499999999999)


In [8]:
save_path = saver.save(sess, model_path, global_step=(epoch+1))
print('the save path is ', save_path)

('the save path is ', '../ckpt/han-bigru-title-content-256/model.ckpt-4')


- Bi-LSTM 模型<br/>
**batch_size=256，迭代12个epoch，基本收敛。结果： 验证集 f1=0.38618； 提交结果 0.3873186**
- Bi-GRU 模型<br/>
**batch_size=256，迭代15个epoch。大概在 13 个 epoch 就收敛了。结果： 验证集 f1=0.390534； 提交结果 0.39198**
- attention-Bi-GRU 模型<br/>
**batch_size=256，迭代18个epoch。在16个epoch收敛。结果：验证集 f1=f1=0.391734 ；提交结果 0.39310**

## 若没有收敛，继续迭代

In [ ]:
_lr = 1e-4
max_max_epoch = 4  # 本例中
add_epoch = 4
decay = 0.85
decay_step = 6000        # 每 valid_step 就进行一次 valid 运算

for epoch in xrange(max_max_epoch, max_max_epoch+add_epoch):
    _lr = _lr * decay
    print 'EPOCH %d， lr=%g' % (epoch+1, _lr)
    time0 = time.time()
    _costs = 0.0
    for batch in xrange(tr_batch_num): 
        global_step += 1
        if (batch+1) % decay_step == 0:
            _lr = _lr * decay
            valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
            print('valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
                    valid_cost, precision, recall, f1, time.time()-time0) )
            print('lr=%g' % _lr)
        X_batch, y_batch = data_train.next_batch(tr_batch_size)
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        y_batch = to_categorical(y_batch)
        feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:_lr, batch_size:tr_batch_size, keep_prob:0.5}
        fetches = [merged, cost, train_op]
        summary, _cost, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        _costs += _cost
        if global_step % 100:
            train_writer.add_summary(summary, global_step)
            X_batch, y_batch = data_valid.next_batch(tr_batch_size)
            X1_batch = X_batch[:, :title_len]
            X2_batch = X_batch[:, title_len:]
            y_batch = to_categorical(y_batch)
            feed_dict = {X1_inputs:X1_batch,X2_inputs:X2_batch, y_inputs:y_batch, lr:1e-5,
                         batch_size:tr_batch_size, keep_prob:1.0}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)
    valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
    mean_cost = _costs / tr_batch_num
    print('training cost=%g; valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
        mean_cost, valid_cost, precision, recall, f1, time.time()-time0) )
    if (epoch + 1) % save_epoch == 0:  # 每 2 个 epoch 保存一次模型
        save_path = saver.save(sess, model_path, global_step=(epoch+1))
        print('the save path is ', save_path)        

EPOCH 5， lr=8.5e-05
valid cost=0.00419687; p=1.37203, r=0.557984, f1=0.396666; speed=6754.84 s/epoch
lr=7.225e-05
training cost=0.00389748; valid cost=0.00422158; p=1.37473, r=0.558969, f1=0.397389; speed=12951 s/epoch
('the save path is ', '../ckpt/han-bigru-title-content-256/model.ckpt-5')
EPOCH 6， lr=6.14125e-05


KeyboardInterrupt: 

## 本地测试

In [3]:
sys.path.append('..')
from evaluator import score_eval

print('begin ...')
# X_valid = np.load('../data/X_valid.npy')
# 保存 本地测试的标注数据
marked_labels_list = np.load('../data/marked_labels_list.npy')
local_scores_path = '../local_scores/' + model_name + '.npy'

# 导入保存好的模型
saver = tf.train.Saver()
best_model_path = model_path + '-' + str(5)  # 导入最优模型
saver.restore(sess, best_model_path)
print('Finished loading model.')

begin ...
INFO:tensorflow:Restoring parameters from ../ckpt/han-bigru-title-content-256/model.ckpt-5
Finished loading model.


In [6]:
local_scores_path = '../local_scores/' + model_name + '.npy'
# 导入测试数据
def local_predict(scores_path=local_scores_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    time0 = time.time()
    X_te = X_valid
    n_sample = X_te.shape[0]  # 测试样本数量
    _batch_size = 500
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in xrange(0, n_sample, _batch_size):
        start = i
        end = start+_batch_size
        if end > n_sample:
            end = n_sample
            _batch_size = end - start
        X_batch = X_te[start:end]
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, lr:1e-5, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step }
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
    np.save(local_scores_path, predict_scores)
    print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    
local_predict()

local predicting ...
local valid p=1.41507, r=0.57667, f1=0.409706; speed=63.837 s/epoch
Writed the scores into ../local_scores/han-cnn-title-content-256-23457-1234.npy, time 64.2625 s


## 对测试数据进行预测

In [15]:
# 导入保存好的模型
saver = tf.train.Saver()
best_model_path = model_path + '-' +str(4)
saver.restore(sess, best_model_path)

result_path = '../result/' + model_name + '-epoch4.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '-epoch4.npy'            # scores.npy 位置

INFO:tensorflow:Restoring parameters from ../ckpt/han-bigru-title-content-256/model.ckpt-4


In [9]:
save_path = '../data/'
# 导入测试数据
def predict():
    """预测测试集结果，并保存到  result.csv 文件中。"""
    X_title = np.load(save_path+'X_te_title_50.npy') 
    X_content = np.load(save_path+'eval_segs_content.npy')
    X_content = X_content.reshape([-1, doc_len*sent_len])
    X_te = np.hstack([X_title, X_content])
    n_sample = X_te.shape[0]  # 测试样本数量
    _batch_size = 500
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in xrange(0, n_sample, _batch_size):
        start = i
        end = start+_batch_size
        if end > n_sample:
            end = n_sample
            _batch_size = end - start
        X_batch = X_te[start:end]
        X1_batch = X_batch[:, :title_len]
        X2_batch = X_batch[:, title_len:]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, lr:1e-5, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    return predict_labels_list, np.vstack(predict_scores)

def write_result(predict_labels_list, result_path):
    """把结果写到 result.csv 中"""
    eval_question = np.load('../data/eval_question.npy')
    with open('../data/sr_topic2id.pkl', 'rb') as inp:
        sr_topic2id = pickle.load(inp)
        sr_id2topic = pickle.load(inp)
    pred_labels = np.asarray(predict_labels_list).reshape([-1])
    pred_topics = sr_id2topic[pred_labels].values.reshape([-1, 5])   # 转为 topic
    df_result = pd.DataFrame({'question':eval_question, 'tid0': pred_topics[:,0], 'tid1':pred_topics[:, 1],
                         'tid2': pred_topics[:,2], 'tid3':pred_topics[:,3],'tid4': pred_topics[:,4]})
    df_result.to_csv(result_path, index=False, header=False)
    print('Finished writing the result')
    return df_result

In [10]:
%time predict_labels_list,predict_scores = predict()
df_result = write_result(predict_labels_list, result_path=result_path) 
print('len(df_result)=',len(df_result))  # 结果应该为 217360
print('Saving the predict_scores into %s' % scores_path)
print('predict_scores.shape=',predict_scores.shape)
np.save(scores_path, predict_scores)
print('Finished saving the result!')

CPU times: user 2min 16s, sys: 16.5 s, total: 2min 33s
Wall time: 2min 30s
Finished writing the result
('len(df_result)=', 217360)
Saving the predict_scores into ../scores/han-cnn-title-content-256-23457-1234.npy
('predict_scores.shape=', (217360, 1999))
Finished saving the result!


In [14]:
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-6839713564940654454,-7506384235581390893,-5932391056759866388,-6306904715218704629
1,6649324930261961840,-240041917918953337,3418451812342379591,2858911571784840089,9149698301250936831,2382911985227044227
2,-4251899610700378615,2919247920214845195,-7358589937244777363,-429636223750539488,2816249700493135244,-5265476641576484497
3,6213817087034420233,-8655945395761165989,-4966205278807386328,5804619920623030604,-383231890572301883,7476760589625268543
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,7951349602759061249,6018641953300645757


In [11]:
# 参考结果
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,-7506384235581390893,2919247920214845195,4610596224687453206,-5932391056759866388,-6306904715218704629
1,6649324930261961840,-240041917918953337,3418451812342379591,2858911571784840089,2382911985227044227,3383016985780045156
2,-4251899610700378615,2919247920214845195,-5265476641576484497,-7358589937244777363,-5385537249890202479,-3315241959305847628
3,6213817087034420233,-8655945395761165989,-4966205278807386328,5804619920623030604,1790630374868647951,7476760589625268543
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,6018641953300645757,-985918886318729368


## 在全部预测正确的情况下，理论值为：f1=0.713933
precision=2.50273, recall=0.998873, f1=0.713933

In [26]:
# 假设全部正确，f1 值最高能到多少
def padding_label(labels):
    """把所有的label补齐到长度为 5"""
    label_len = len(labels)
    if label_len >= 5:
        return labels[:5]
    return np.hstack([labels, np.zeros(5-label_len, dtype=int) - 1])
    

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
predict_labels_list = map(padding_label, marked_labels_list)
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print '在全部预测正确的情况下，理论值为：'
print 'precision=%g, recall=%g, f1=%g' % (precision, recall, f1)

在全部预测正确的情况下，理论值为：
precision=2.50273, recall=0.998873, f1=0.713933
